In [1]:
!pip install spacy
!pip install textblob
!pip install nltk
!pip install tqdm
!pip install pandas openpyxl xlsxwriter 
!pip install numpy 
!pip install torch transformers
!pip install scikit-learn
!pip install plotly
!pip install streamlit 
!pip install nbformat 
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 682.7 kB/s eta 0:00:19
     --------------------------------------- 0.0/12.8 MB 445.2 kB/s eta 0:00:29
     --------------------------------------- 0.1/12.8 MB 440.4 kB/s eta 0:00:29
     --------------------------------------- 0.1/12.8 MB 440.4 kB/s eta 0:00:29
     --------------------------------------- 0.1/12.8 MB 440.4 kB/s eta 0:00:29
     --------------------------------------- 0.1/12.8 MB 437.6 kB/s eta 0:00:30
     --------------------------------------- 0.1/12.8 MB 425.1 kB/s eta 0:00:30
     --------------------------------------- 0.1/12.8 MB 426.7 kB/s eta 0:00:30
      -------------------------------------- 0.2/12.8 MB 456.4 kB/s eta 0:00:28
      -------------------------------------- 0.2/12.8 MB 456.4 kB/s eta 0:00:28
      -------------------------------------- 0.2/12.8 MB 445.2 kB/s eta 0:00:29
      -------------------------------------- 0.2

In [2]:
import pandas as pd
import numpy as np
import os
import re
import glob
import textblob
import torch
import streamlit as sl
import plotly.express as px

import nltk
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

import spacy as sp
nlp_s = sp.load("en_core_web_sm")

from textblob import TextBlob
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer

from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
sia = SentimentIntensityAnalyzer()

os.chdir("c:/Users/mabra/OneDrive/Pulpit/Studia/BIG DATA/SMESTR 3/Web Mining/Project")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mabra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\mabra\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\mabra\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mabra\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mabra\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
c:\Users\mabra\anaconda3\envs\temp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidget

In [3]:
table = [
    {'selector': 'th',
     'props': [('background-color', '#31A9FF'),
               ('color', 'black'),
               ('border-color', 'black'),
               ('border-style', 'solid'),
               ('border-width', '1px')]},
    {'selector': 'td',
     'props': [('border-color', 'black'),
               ('border-style', 'solid'),
               ('border-width', '1px'),
               ('background-color', '#C4EBF4'),
               ('color', 'black')]},
    {'selector': 'tr:nth-child(even) td',
     'props': [('background-color', 'white')]},
]

In [4]:
# Creating data frame that contains all of the comments
path = os.getcwd()
files = os.listdir(path)
# Creating dataframe that contains all of the posts
post_pattern = re.compile(r".*posts\.csv$")
poss = [file for file in files if post_pattern.match(file)]

posts = pd.DataFrame()

for p in poss:
    data = pd.read_csv(p, encoding='latin')
    posts = pd.concat([posts,data], ignore_index=True)

# Creating dataframe for tickers of 7317 companys
tickers = pd.read_csv('nasdaq.csv')

In [5]:
print('posts before removing duplictes', len(posts))
posts = posts.drop_duplicates(subset="id")
print('posts after removing duplictes: ', len(posts))
print('The number of different stocks which will be monitored: ', len(tickers))

posts before removing duplictes 141130
posts after removing duplictes:  137286
The number of different stocks which will be monitored:  7317


In [6]:
posts.head(4).style.set_table_styles(table)

,id,url,title,score,num_comments,created_utc,selftext,name,author,link_flair_css_class,link_flair_text
0,j6ekeb,https://i.redd.it/imwz9oh2rjr51.jpg,Theory:PRPL stuff up because opposite of orange man on color wheel,1,0,1602021554.000000,[deleted],t3_j6ekeb,nan,shitpost,Shitpost
1,j6ek9b,https://i.redd.it/vo82me31rjr51.jpg,Ive seen this movie before!,1,0,1602021541.000000,nan,t3_j6ek9b,Clydian_Fayre,shitpost,Shitpost
2,j6ej5z,https://www.reddit.com/r/wallstreetbets/comments/j6ej5z/was_just_sleeping_my_cave_for_the_past_few_months/,Was just sleeping my ???? cave for the past few months. Today around 2:45pm i was awoken by a loud crashing sound.,1,1,1602021441.000000,[removed],t3_j6ej5z,nan,question,Discussion
3,j6ehe9,https://www.reddit.com/r/wallstreetbets/comments/j6ehe9/noob/,Noob,1,1,1602021273.000000,[removed],t3_j6ehe9,DapperNewspaper,question,Discussion


In [7]:
posts_title = posts
posts["date"] = pd.to_datetime(posts["created_utc"], unit="s")
posts_title["title_tokens"] = posts_title["title"].apply(word_tokenize)

def stocks(row):
    return any(word in tickers["Symbol"].tolist() for word in row)

posts_title = posts_title[posts_title["title_tokens"].apply(stocks)]

In [8]:
# Removing tickers which had only one letter since after review most of them seem to be a mistake for example "A" was often used as
# part of english vocabulary when people were writing in all caplslog 

# After removing Stocks that have only one letter i analyzed posts that were in top 30 and discarded shortcuts that did not
# acctually refered to any comapnys or they were used commonly as a word rather then company shortcut
new_tickers = [word for word in tickers["Symbol"] if isinstance(word, str) and len(word) > 1]
new_tickers = [word for word in new_tickers if word != "DD"] # Shortcut for Daily Discussion
new_tickers = [word for word in new_tickers if word != "HAS"] # common word
new_tickers = [word for word in new_tickers if word != "TD"] # Shortcut for TD Ameritrade
new_tickers = [word for word in new_tickers if word != "YOU"] # common word
new_tickers = [word for word in new_tickers if word != "RH"] # Shortcut for Robin Hood
new_tickers = [word for word in new_tickers if word != "ALL"] # Common word
new_tickers = [word for word in new_tickers if word != "ON"] # Common word
new_tickers = [word for word in new_tickers if word != "UK"] # Shortcut for Unighted Kingdom
new_tickers = [word for word in new_tickers if word != "UP"] # Common word
new_tickers = [word for word in new_tickers if word != "FOR"]
len(new_tickers)

7285

In [9]:
def stocks(row):
    f_ticker = next((word for word in row if word in new_tickers), None)
    return pd.Series({'found_ticker': f_ticker})

posts_title["found_ticker"] = posts_title.apply(lambda row: stocks(row["title_tokens"]), axis=1)

C:\Users\mabra\AppData\Local\Temp\ipykernel_14788\2889341875.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts_title["found_ticker"] = posts_title.apply(lambda row: stocks(row["title_tokens"]), axis=1)


In [10]:
posts_title.head(3)

,id,url,title,score,num_comments,created_utc,selftext,name,author,link_flair_css_class,link_flair_text,date,title_tokens,found_ticker
0,j6ekeb,https://i.redd.it/imwz9oh2rjr51.jpg,Theory:PRPL stuff up because opposite of orang...,1,0,1.602022e+09,[deleted],t3_j6ekeb,NaN,shitpost,Shitpost,2020-10-06 21:59:14,"[Theory, :, PRPL, stuff, up, because, opposite...",PRPL
9,j6ecr1,https://v.redd.it/5rni8l8iojr51,"$500 to $750 $D 80c 1/15/21, 87.50c 4/16/21",1,0,1.602021e+09,[deleted],t3_j6ecr1,NaN,profit,Gain,2020-10-06 21:47:04,"[$, 500, to, $, 750, $, D, 80c, 1/15/21, ,, 87...",None
17,j6e3mv,https://i.redd.it/qk8qshydmjr51.jpg,I just got a tattoo of every reason to invest ...,1,0,1.602020e+09,[deleted],t3_j6e3mv,NaN,shitpost,Shitpost,2020-10-06 21:33:00,"[I, just, got, a, tattoo, of, every, reason, t...",NKLA


In [11]:
# Creating table for number of tickers per month and their numbers in posts titles
posts_title["month"] = posts_title["date"].dt.month
posts_title_table = posts_title.groupby(["month","found_ticker"]).size().unstack(fill_value=0)
posts_title_table = posts_title_table.iloc[1:]
# Creating table that shows the % amount of each stock relative to other in each month
posts_title_table_per_row = posts_title_table.div(posts_title_table.sum(axis=1), axis=0) * 100
# Creating table that will show the percentage change in disccusion of the stock for X mot popular stocks
posts_title_table_per_col = posts_title_table.iloc[1:].pct_change()

# Creating table for unique users per month (Table for all posts not only stock ones)
posts_users_table = posts
posts_users_table["month"] = posts_users_table["date"].dt.month
posts_users_table = posts_users_table.fillna("[removed]")
posts_users_table = posts_users_table[~posts_users_table["author"].str.contains(r'\[removed\]|\[deleted\]|^$')]
posts_users_table = posts_users_table.groupby('month')['author'].nunique()
posts_users_table = posts_users_table.iloc[1:]

# Creating table for unique users posting about stocks per month
posts_users_table_stocks = posts_title
posts_users_table_stocks["month"] = posts_users_table_stocks["date"].dt.month
posts_users_table_stocks = posts_users_table_stocks.fillna("[removed]")
posts_users_table_stocks = posts_users_table_stocks[~posts_users_table_stocks["author"].str.contains(r'\[removed\]|\[deleted\]|^$')]
posts_users_table_stocks = posts_users_table_stocks.groupby('month')['author'].nunique()
posts_users_table_stocks = posts_users_table_stocks.iloc[1:]

# Creating sum of top 30 mentioned stocks and sum of all
top_30_sums = []
all_sums = []

# Iterating over each month
for index, row in posts_title_table.iterrows():
    # Sort the values in descending order and select top 20
    top_30 = row.sort_values(ascending=False)[:30]
    all_s = row.sum()
    all_sums.append(all_s)

    # Sum these top 20 values and add to the list
    top_30_sum = top_30.sum()
    top_30_sums.append(top_30_sum)

top_in_all = pd.DataFrame({'top_30': top_30_sums, 'all': all_sums}, index= posts_users_table.index)

# Creating table for monthly active unique users
# Creating percent change for daily active users for posts_users_table and posts_users_table_stocks
posts_users_table_per = posts_users_table.pct_change()

posts_users_table_stocks_per = posts_users_table_stocks.pct_change()

# Table for most popular stock each month
stock_of_the_month = posts_title_table.idxmax(axis=1)

monthly_dfs = []

for month in posts_title_table.index:
    sorted_df = posts_title_table.sort_values(by=month, axis=1, ascending=False)
    top_30 = sorted_df.iloc[:,:30]
    top_30_values = list(top_30.loc[month])
    top_30_tickers = list(top_30.columns)
    month_df = pd.DataFrame({'tickers': top_30_tickers, 'values': top_30_values, 'month': month})
    monthly_dfs.append(month_df)

monthly_graphs = pd.concat(monthly_dfs, ignore_index=True)

# list with month names
month_number = [1,2,3,4,5,6,7,8,9,10,11,12]
month_name = ['January','February','March','April','May','June','July','August','September','October','November','December']
month_names = pd.DataFrame({'number': month_number,
                            'name': month_name})

# List for top 30 of all time
top_30_all_time = pd.DataFrame(posts_title_table.sum().sort_values(ascending=False)[:30])

C:\Users\mabra\AppData\Local\Temp\ipykernel_14788\2071590460.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts_title["month"] = posts_title["date"].dt.month
C:\Users\mabra\AppData\Local\Temp\ipykernel_14788\2071590460.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts_users_table_stocks["month"] = posts_users_table_stocks["date"].dt.month


In [12]:
top_in_all

,top_30,all
month,,
6,1417,3249
7,2082,3564
8,2345,4062
9,2312,3564
10,1494,2653
11,6611,9202
12,6240,9843


In [13]:
posts_users_table

month
6      6199
7      5387
8      5780
9      6070
10     4819
11     9096
12    11786
Name: author, dtype: int64

In [14]:
top_30_sums = []
all_sums = []

# Iterating over each month
for index, row in posts_title_table.iterrows():
    # Sort the values in descending order and select top 20
    top_30 = row.sort_values(ascending=False)[:30]
    all_s = row.sum()
    all_sums.append(all_s)

    # Sum these top 20 values and add to the list
    top_30_sum = top_30.sum()
    top_30_sums.append(top_30_sum)


In [15]:
posts_title[posts_title["found_ticker"]=="QS"]

,id,url,title,score,num_comments,created_utc,selftext,name,author,link_flair_css_class,link_flair_text,date,title_tokens,found_ticker,month
22201,k41awa,https://www.reddit.com/r/wallstreetbets/commen...,Is QS a buy?,2,0,1.606760e+09,[removed],t3_k41awa,doug_diablo,question,Discussion,2020-11-30 18:05:27,"[Is, QS, a, buy, ?]",QS,11
22240,k40pep,https://www.reddit.com/r/wallstreetbets/commen...,$QS DDGet your short shorts boys.,14,74,1.606758e+09,The tendie train of the day is QuantomScape ($...,t3_k40pep,halebass,dd,DD,2020-11-30 17:40:10,"[$, QS, DDGet, your, short, shorts, boys, .]",QS,11
22709,k3xyxx,https://www.barrons.com/articles/ev-battery-ma...,??QS is making the Barron's Bros shit themselv...,2,0,1.606750e+09,[deleted],t3_k3xyxx,NaN,stocks,Stocks,2020-11-30 15:32:09,"[?, ?, QS, is, making, the, Barron, 's, Bros, ...",QS,11
22870,k3x8ek,https://www.reddit.com/r/wallstreetbets/commen...,Battery startup backed by VW and Tesla cofound...,1,0,1.606748e+09,[removed],t3_k3x8ek,WrongChoices,stocks,Stocks,2020-11-30 14:54:15,"[Battery, startup, backed, by, VW, and, Tesla,...",QS,11
22913,k3x0dh,https://i.redd.it/zk2pcfu83e261.jpg,QuantumScape $QS - The commercially viable Sol...,1,0,1.606747e+09,[deleted],t3_k3x0dh,NaN,meme,Meme,2020-11-30 14:42:17,"[QuantumScape, $, QS, -, The, commercially, vi...",QS,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69535,kd81wc,https://i.redd.it/tnnsmx7w98561.jpg,GUYS HELP?! Words are free lol. Just tell me w...,1,0,1.607984e+09,[deleted],t3_kd81wc,NaN,loss,Loss,2020-12-14 22:21:16,"[GUYS, HELP, ?, !, Words, are, free, lol, ., J...",QS,12
70080,kcydjy,https://www.reddit.com/r/wallstreetbets/commen...,"Sell QuantumScape (QS), secure profits, and re...",26,18,1.607955e+09,Sell QuantumScape (QS) and secure profits!\n\n...,t3_kcydjy,Torlek1,question,Discussion,2020-12-14 14:14:23,"[Sell, QuantumScape, (, QS, ), ,, secure, prof...",QS,12
70569,kcf6ke,https://www.reddit.com/r/wallstreetbets/commen...,Quantum Scape (QS) Short Squeeze Candidate ??????,1,0,1.607882e+09,[removed],t3_kcf6ke,jagjr712,question,Discussion,2020-12-13 17:59:31,"[Quantum, Scape, (, QS, ), Short, Squeeze, Can...",QS,12
70681,kc7pfx,https://www.reddit.com/r/wallstreetbets/commen...,Quantum Scape is about to take a Quantum Dump ...,116,123,1.607851e+09,"Alright you *shrimp-dick fuck nuggets*, listen...",t3_kc7pfx,QVonesh,dd,DD,2020-12-13 09:20:36,"[Quantum, Scape, is, about, to, take, a, Quant...",QS,12


In [65]:
with pd.ExcelWriter('data_streamlit.xlsx', engine='xlsxwriter') as data_streamlit:
    posts_title_table.to_excel(data_streamlit, sheet_name='posts_title_table')
    posts_title_table_per_row.to_excel(data_streamlit, sheet_name='posts_title_table_per_row')
    posts_title_table_per_col.to_excel(data_streamlit, sheet_name='posts_title_table_per_col')
    posts_users_table.to_excel(data_streamlit, sheet_name='posts_users_table')
    posts_users_table_stocks.to_excel(data_streamlit, sheet_name='posts_users_table_stocks')
    posts_users_table_per.to_excel(data_streamlit, sheet_name='posts_users_table_per')
    posts_users_table_stocks_per.to_excel(data_streamlit, sheet_name='posts_users_table_stocks_per')
    monthly_graphs.to_excel(data_streamlit, sheet_name='monthly_graphs')
    month_names.to_excel(data_streamlit, sheet_name='month_names')
    top_in_all.to_excel(data_streamlit, sheet_name='top_in_all')

In [97]:
# Table that is showing change in the frequency of the stock name
posts_title_table

found_ticker,AA,AAL,AAOI,AAP,AAPL,AAU,ABBV,ABCL,ABEV,ABIO,...,ZBRA,ZDGE,ZI,ZKIN,ZM,ZOM,ZS,ZTO,ZUO,ZYXI
month,,,,,,,,,,,,,,,,,,,,,
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,5,75,0,0,21,0,5,0,0,0,...,0,0,3,0,47,3,1,0,4,0
7,3,7,0,0,46,0,6,0,0,0,...,0,0,2,0,5,1,1,0,0,0
8,2,16,0,3,204,1,0,0,0,2,...,0,3,2,0,8,0,1,0,1,1
9,1,11,0,0,176,0,0,0,0,0,...,0,0,2,0,51,0,3,0,0,0
10,1,5,0,0,82,0,2,0,0,0,...,0,0,0,0,13,1,0,0,0,0
11,2,7,0,0,38,0,3,1,0,0,...,0,0,4,2,39,1,0,0,1,0
12,2,11,2,0,108,0,2,13,2,0,...,2,0,2,0,15,1,2,1,10,0
